## Scraping skatingscores.com

The purpose of this notebook is to test and then sucesfully scrape https://skatingscores.com/. Practically, I could ask for the data, but because this is a learning project we will scrape assuming we couldn't get it.

In [244]:
#pip install lxml html5lib beautifulsoup4
import pandas as pd
import numpy as np

import requests
from bs4 import BeautifulSoup
import time

Better data of what I want
- each skater can be described by a set of 11 tables


https://skatingscores.com/2021/wc/sr/women/long/

Called the "Enhanced Protocol" according to skatingscores

In [186]:
# #world championship 2021, women's long
# wc2021_wl = scrapeEvent(website = 'https://skatingscores.com/2021/wc/sr/women/long/')
# wc2021_wl

In [208]:
class scrapeEvent:
    """
    Class for scraping the "Enhanced Protocol" scoresheet from skatingscores.com, initates by recording all skaders
    """
    def __init__(self, website, event_name):

        self.event_name = event_name
        self.dfs = pd.read_html(website)

        self.start = 0
        self.n = len(self.dfs)
        self.stop = self.n - 2
        self.sets = []

        start = self.start
        stop = self.stop

        all_skaters = []
        while start <= stop:
            left = start
            start += 11
            right = start

            #bio is located in first dataframe in sets of 11
            raw_bio = self.dfs[left:right][1].iloc[:, 1:] 
            raw_bio.columns = raw_bio.iloc[0,:]
            bio = raw_bio.iloc[1:,:].rename(columns={raw_bio.columns[0]: "Name", raw_bio.columns[1]: "Country"})
            all_skaters.append(bio)

        all_skaters = pd.concat(all_skaters, axis = 0).reset_index(drop = True)
        all_skaters['Event_Name'] = event_name
        self.all_skaters = all_skaters
                    
        time.sleep(0.33) # be nice to the generous server admins!

    def get_component_score(self, drop_rank = True):
        """
        Aggregates all component scores by skater 
        """
        start = self.start
        stop = self.stop

        all_components = []
        skater_count = 0

        while start <= stop:
            left = start
            start += 11
            right = start

            #bio is located in TENTH dataframe in sets of 11
            raw_component = self.dfs[left:right][10].iloc[:, 1:] 
            raw_component.columns = raw_component.iloc[0,:]
            raw_component = raw_component.loc[:,~raw_component.columns.duplicated()]

            component = raw_component.iloc[1:-1, :-1]
            component.index = component['Component']
            component.drop(columns = 'Component', inplace = True)

            component.insert(0, 'Name', self.all_skaters['Name'][skater_count])
            component.insert(1, 'Country', self.all_skaters['Country'][skater_count])
            skater_count += 1     

            all_components.append(component)

        all_components = pd.concat(all_components, axis = 0)
        for c in all_components.loc[:, ~(all_components.columns.isin(['Name', 'Country', 'Factor']))].columns:
            all_components[c] = all_components[c].str.split(' ').str[1]

        all_components['Event'] = self.event_name
        self.all_components = all_components



    def get_technical_score(self):
        """
        Aggregates all technical scores by skater
        """
        start = self.start
        stop = self.stop

        all_technical = []
        skater_count = 0

        while start <= stop:
            left = start
            start += 11
            right = start
        
         #bio is located in THIRD dataframe in sets of 11
            raw_technical = self.dfs[left:right][3].iloc[:, 1:] 
            raw_technical = raw_technical.dropna(axis = 1)
            raw_technical.columns = raw_technical.iloc[0,:] #use 1st row as column
            technical = raw_technical.iloc[1:-1, :] # remove last row which contains column sums and the first row containing columns names

            technical.insert(0, 'Name', self.all_skaters['Name'][skater_count])
            technical.insert(1, 'Country', self.all_skaters['Country'][skater_count])
            skater_count += 1     

            all_technical.append(technical)

        all_technical = pd.concat(all_technical, axis = 0).reset_index(drop = True)
        for i in all_technical.columns[3:]:
            all_technical[i] = all_technical[i].astype(float, errors = 'ignore')

        all_technical['Event'] = self.event_name
        self.all_technical = all_technical

wc2021_wl = scrapeEvent(website = 'https://skatingscores.com/2122/crncup/sr/women/long/', event_name = "World Championship Women")

In [209]:
wc2021_wl.get_component_score()
wc2021_wl.all_components

,Name,Country,Factor,J1 🇲🇽,J2 🇹🇷,J3 🇨🇦,J4 🇺🇸,J5 🇸🇪,Event
Component,,,,,,,,,
Skating Skills,Alysa LIU,USA,1.60,7.50,7.25,8.50,7.75,7.75,World Championship Women
Transitions,Alysa LIU,USA,1.60,7.50,7.00,8.00,8.25,8.00,World Championship Women
Performance,Alysa LIU,USA,1.60,7.25,7.50,8.50,8.50,8.50,World Championship Women
Composition,Alysa LIU,USA,1.60,8.00,7.00,8.50,9.00,8.25,World Championship Women
Interpretation,Alysa LIU,USA,1.60,8.00,7.25,8.25,8.75,8.25,World Championship Women
...,...,...,...,...,...,...,...,...,...
Skating Skills,Andrea MONTESINOS CANTÚ,MEX,1.60,6.50,5.75,5.50,4.75,4.75,World Championship Women
Transitions,Andrea MONTESINOS CANTÚ,MEX,1.60,6.25,5.50,4.50,4.00,4.25,World Championship Women
Performance,Andrea MONTESINOS CANTÚ,MEX,1.60,5.75,5.50,4.25,4.25,4.50,World Championship Women


In [210]:
wc2021_wl.get_technical_score()
wc2021_wl.all_technical

,Name,Country,Element,BaseValue,GOETotal,J1 🇲🇽,J2 🇹🇷,J3 🇨🇦,J4 🇺🇸,J5 🇸🇪,Event
0,Alysa LIU,USA,3A<<,3.30,-1.65,-5.0,-5.0,-5.0,-5.0,-5.0,World Championship Women
1,Alysa LIU,USA,3Lz,5.90,1.18,1.0,2.0,2.0,2.0,2.0,World Championship Women
2,Alysa LIU,USA,3Lo,4.90,0.82,1.0,2.0,2.0,1.0,3.0,World Championship Women
3,Alysa LIU,USA,2A+1Eu+3S,8.10,0.86,2.0,2.0,2.0,1.0,3.0,World Championship Women
4,Alysa LIU,USA,FCCoSp4,3.50,0.93,2.0,2.0,4.0,3.0,3.0,World Championship Women
...,...,...,...,...,...,...,...,...,...,...,...
247,Andrea MONTESINOS CANTÚ,MEX,2A+1Eu+2S,5.61,0.44,1.0,2.0,1.0,2.0,1.0,World Championship Women
248,Andrea MONTESINOS CANTÚ,MEX,2A,3.63,-1.65,-5.0,-5.0,-5.0,-5.0,-5.0,World Championship Women
249,Andrea MONTESINOS CANTÚ,MEX,2F,1.98,0.24,1.0,2.0,2.0,1.0,0.0,World Championship Women
250,Andrea MONTESINOS CANTÚ,MEX,ChSq117,3.00,0.33,1.0,1.0,1.0,-1.0,0.0,World Championship Women


todo

-element needs to be cleaned up because it contains annotations (ex: Elemeent = 3Aq which denotes a triple axel, but something is wrong with it)

In [ ]:
# dfs = pd.read_html('https://skatingscores.com/2021/wc/sr/women/long/')
# for i in np.arange(13):
#     print(i)
#     display(dfs[i].head(), dfs[i].shape)

## Scraping the events page

In [211]:
events2122_raw = pd.read_html('https://skatingscores.com/2122/')[1]
events2122_raw.columns = ['Country', 'Event_Abbreviation', 'Event_Name', 'Event_Begin']
events2122_raw

,Country,Event_Abbreviation,Event_Name,Event_Begin
0,🇺🇸,CRNCUP,Cranberry Cup,"Aug 11, 2021"
1,🇺🇸,LPIDI,Lake Placid Ice Dance Intl,"Aug 12, 2021"
2,🇫🇷,JGPFRA,JGP de Courchevel I,"Aug 18, 2021"
3,🇨🇦,JGPCAN,JGP Canada,"Aug 25, 2021"
4,🇫🇷,JGPFRA2,JGP de Courchevel II,"Aug 25, 2021"
...,...,...,...,...
109,🇧🇬,WJC,World Jr Championships,"Mar 07, 2022"
110,🇱🇺,COUPRI,Coupe du Printemps,"Mar 18, 2022"
111,🇸🇮,TRITRO,Triglav Trophy,"Mar 18, 2022"
112,🇫🇷,WC,World Championships,"Mar 21, 2022"


In [189]:
## CODE CREDIT GOES TO https://stackoverflow.com/questions/56757261/extract-href-using-pandas-read-html, the work below has been modified and is not fully my own

url = 'https://skatingscores.com/2122/'
df = pd.read_html(url)[1]

response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')
soup.prettify()
tables = soup.find_all("tbody")

links = []
for tr in tables[1].findAll("tr"):
    trs = tr.findAll("td")
    for each in trs:
        try:
            link = each.find('a')['href']
            links.append(link)
        except:
            pass
links_to_events = [url + i[6:] for i in links]
abbrv_of_valid_events = [i[len(url):-1].upper() for i in links_to_events] # some events were cancelled and are not pulled via this method above
links_to_completed_events = pd.DataFrame()
links_to_completed_events['Abbreviation'] = abbrv_of_valid_events
links_to_completed_events['URL'] = links_to_events
links_to_completed_events

,Abbreviation,URL
0,CRNCUP,https://skatingscores.com/2122/crncup/
1,LPIDI,https://skatingscores.com/2122/lpidi/
2,JGPFRA,https://skatingscores.com/2122/jgpfra/
3,JGPFRA2,https://skatingscores.com/2122/jgpfra2/
4,JGPSVK,https://skatingscores.com/2122/jgpsvk/
...,...,...
76,NATUSA,https://skatingscores.com/2122/natusa/
77,NATCAN,https://skatingscores.com/2122/natcan/
78,NATKOR,https://skatingscores.com/2122/natkor/
79,EC,https://skatingscores.com/2122/ec/


In [190]:
events_2122 = pd.merge(left = events2122_raw, right = links_to_completed_events, left_on = 'Event_Abbreviation', right_on = 'Abbreviation', how = 'inner')
events_2122 

,Country,Event_Abbreviation,Event_Name,Event_Begin,Abbreviation,URL
0,🇺🇸,CRNCUP,Cranberry Cup,"Aug 11, 2021",CRNCUP,https://skatingscores.com/2122/crncup/
1,🇺🇸,LPIDI,Lake Placid Ice Dance Intl,"Aug 12, 2021",LPIDI,https://skatingscores.com/2122/lpidi/
2,🇫🇷,JGPFRA,JGP de Courchevel I,"Aug 18, 2021",JGPFRA,https://skatingscores.com/2122/jgpfra/
3,🇫🇷,JGPFRA2,JGP de Courchevel II,"Aug 25, 2021",JGPFRA2,https://skatingscores.com/2122/jgpfra2/
4,🇸🇰,JGPSVK,JGP Slovakia,"Sep 01, 2021",JGPSVK,https://skatingscores.com/2122/jgpsvk/
...,...,...,...,...,...,...
77,🇺🇸,NATUSA,US Nationals,"Jan 04, 2022",NATUSA,https://skatingscores.com/2122/natusa/
78,🇨🇦,NATCAN,Canadian Nationals,"Jan 07, 2022",NATCAN,https://skatingscores.com/2122/natcan/
79,🇰🇷,NATKOR,Korean Nationals,"Jan 07, 2022",NATKOR,https://skatingscores.com/2122/natkor/
80,🇪🇪,EC,European Championships,"Jan 10, 2022",EC,https://skatingscores.com/2122/ec/


In [ ]:
def main(story_urls):
    t0 = time.time()
    download_stories(story_urls)
    t1 = time.time()
    print(f"{t1-t0} seconds to download {len(story_urls)} stories.")

In [243]:
import concurrent.futures

In [254]:
def main(event_name, website):
    event = scrapeEvent(event_name = event_name, website = website)
    event.get_component_score()
    ALL_COMPONENTS.append(event.all_components)
    event.get_technical_score()
    ALL_TECHNICAL.append(event.all_technical)
    ALL_SKATER.append(event.all_skaters)


In [ ]:
def download_url(url):
    print(url)
    resp = requests.get(url)
    title = "".join(x for x in url if x.isalpha()) + "html"
    
    with open(title, "wb") as fh:
        fh.write(resp.content)
        
    time.sleep(0.25)
        
def download_stories(story_urls):
    for url in story_urls:
        download_url(url)

def main(story_urls):
    t0 = time.time()
    download_stories(story_urls)
    t1 = time.time()
    print(f"{t1-t0} seconds to download {len(story_urls)} stories.")

In [ ]:
import concurrent.futures

MAX_THREADS = 30

def download_url(url):
    print(url)
    resp = requests.get(url)
    title = ''.join(x for x in url if x.isalpha()) + "html"
    
    with open(title, "wb") as fh:
        fh.write(resp.content)
        
    time.sleep(0.25)
    
def download_stories(story_urls):
    threads = min(MAX_THREADS, len(story_urls))
    
    with concurrent.futures.ThreadPoolExecutor(max_workers=threads) as executor:
        executor.map(download_url, story_urls)

def main(story_urls):
    t0 = time.time()
    download_stories(story_urls)
    t1 = time.time()
    print(f"{t1-t0} seconds to download {len(story_urls)} stories.")




In [255]:
DIVISION = ['sr']
SEX = ['women', 'men']
PROGRAM = ['long', 'short']
ending = 'sr/women/long/'


ALL_COMPONENTS = []
ALL_TECHNICAL = []
ALL_SKATER = []
for index, row in events_2122[0:3][['Event_Abbreviation','URL']].iterrows():
    for s in SEX:
        for p in PROGRAM:
            ending = f'sr/{s}/{p}'
            try:
                main(event_name = row['Event_Abbreviation'],website = row['URL'] + ending)
                time.sleep(0.33) # be nice to the generous server admins!
            except Exception as e:
                print(e,row['Event_Abbreviation'], row['URL'])
                pass

HTTP Error 404: Not Found LPIDI https://skatingscores.com/2122/lpidi/
HTTP Error 404: Not Found LPIDI https://skatingscores.com/2122/lpidi/
HTTP Error 404: Not Found LPIDI https://skatingscores.com/2122/lpidi/
HTTP Error 404: Not Found LPIDI https://skatingscores.com/2122/lpidi/
HTTP Error 404: Not Found JGPFRA https://skatingscores.com/2122/jgpfra/
HTTP Error 404: Not Found JGPFRA https://skatingscores.com/2122/jgpfra/
HTTP Error 404: Not Found JGPFRA https://skatingscores.com/2122/jgpfra/
HTTP Error 404: Not Found JGPFRA https://skatingscores.com/2122/jgpfra/


errors can occur because
- event got cancelled (from the panini)
- event may be juniors only (out of scope)


todo
- figure out how to use logging to clean up some outputs
- multiprocessing?


In [256]:
components = pd.concat(ALL_COMPONENTS, axis = 0)
components

,Name,Country,Factor,J1 🇲🇽,J2 🇹🇷,J3 🇨🇦,J4 🇺🇸,J5 🇸🇪,Event,J1 🇨🇦,J2 🇳🇱,J3 🇲🇽,J4 🇹🇷,J5 🇺🇸
Component,,,,,,,,,,,,,,
Skating Skills,Alysa LIU,USA,1.60,7.50,7.25,8.50,7.75,7.75,CRNCUP,NaN,NaN,NaN,NaN,NaN
Transitions,Alysa LIU,USA,1.60,7.50,7.00,8.00,8.25,8.00,CRNCUP,NaN,NaN,NaN,NaN,NaN
Performance,Alysa LIU,USA,1.60,7.25,7.50,8.50,8.50,8.50,CRNCUP,NaN,NaN,NaN,NaN,NaN
Composition,Alysa LIU,USA,1.60,8.00,7.00,8.50,9.00,8.25,CRNCUP,NaN,NaN,NaN,NaN,NaN
Interpretation,Alysa LIU,USA,1.60,8.00,7.25,8.25,8.75,8.25,CRNCUP,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Skating Skills,Adonis Wai Chung WONG,HKG,1.00,NaN,NaN,NaN,NaN,NaN,CRNCUP,5.25,5.25,4.00,6.00,4.75
Transitions,Adonis Wai Chung WONG,HKG,1.00,NaN,NaN,NaN,NaN,NaN,CRNCUP,4.75,4.00,3.00,4.75,4.75
Performance,Adonis Wai Chung WONG,HKG,1.00,NaN,NaN,NaN,NaN,NaN,CRNCUP,4.75,5.00,3.00,5.25,5.00


In [241]:
# technicals = pd.concat(ALL_TECHNICAL, ignore_index = True)
# technicals